In [1]:
def to_categorical(y):
    from keras import backend as k
    return k.one_hot(y, 512)

In [2]:
from keras.layers import Input, Dense, Concatenate,Lambda,Reshape,Dropout
from keras.models import Model
import numpy as np

vector = Input(shape=(512,), name='Vectors-Input',dtype='float32')
vector_shiter = Dense(512)(vector)
position = Input(shape=(1,), name='Positions-Input',dtype='int32')
categorical_position=Lambda(to_categorical, name='Positional_encoding')(position)
reshaped_categorical_position=Reshape((512,))(categorical_position)
concatenated_encoder_input=Concatenate()([vector,reshaped_categorical_position])
encoder_input_divider1 = Dense(1024, name='Encoder-Output-Divider-1',activation='selu')(concatenated_encoder_input)
encoder_input_dropout1=Dropout(0.1, name='Encoder-Output-Dropout-1')(encoder_input_divider1)
encoder_input_divider2 = Dense(512, name='Encoder-Output-Divider-2',activation='selu')(encoder_input_dropout1)
encoder_input_dropout2=Dropout(0.1, name='Encoder-Output-Dropout-2')(encoder_input_divider2)
concatenated_encoder_input1=Concatenate()([vector_shiter,encoder_input_dropout2])
encoder_input_divider3 = Dense(256, name='Encoder-Output-Divider-3',activation='selu')(concatenated_encoder_input1)
encoder_input_dropout3=Dropout(0.1, name='Encoder-Output-Dropout-3')(encoder_input_divider3)
output_vector = Dense(100, name='Encoder-Output',activation='selu')(encoder_input_dropout3)
output_vector2 = Dense(100, name='Encoder-Garbedge',activation='selu')(encoder_input_dropout3)

model = Model(inputs=[vector,position],outputs=[output_vector,output_vector2])
model.compile(
    optimizer='adam',
    loss='mean_absolute_error',
)

Using TensorFlow backend.
C:\Users\DNS\Anaconda3\envs\decoder\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\DNS\Anaconda3\envs\decoder\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\DNS\Anaconda3\envs\decoder\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\DNS\Anaconda3\envs\decoder\lib\site-pa

In [3]:
from bpemb import BPEmb

bpemb_ru = BPEmb(lang="ru", dim=100, vs=3000)

def textreader(st):
    texts = []
    with open("C:/Users/DNS/Desktop/Progi/Python/NeuralNetworks/Курсач/data/np_rev.txt", "r", encoding='utf-8') as fr:
        i=0
        for line in fr:
            i+=1
            if i<st:      
                continue
            texts.append(line.replace('\n',''))
            if i==st+499:
                break
    return texts

def word_vectors_creator(data):
    kol= np.zeros(shape=(len(data)))
    pieces=[]
    for i in range(len(data)):
        a = bpemb_ru.encode(data[i])
        s = bpemb_ru.encode_ids(data[i])
        b =[]
        for f in range(len(a)):
            if s[f] != 0:
                b.append(a[f])
        kol[i]+=len(b)
        pieces.append(b)
    kol = kol.astype(np.int)
    out = np.zeros(shape=(np.sum(kol),100))
    out2 = np.zeros(shape=(np.sum(kol),100))
    buf = 0
    for g in range(len(data)):        
        for k in range(0,kol[g]):
            out[buf]+=np.asarray(bpemb_ru.emb.get_vector(pieces[g][k]))
            for m in range(kol[g]):
                if m!=g:
                    out2[buf]+=(np.asarray(bpemb_ru.emb.get_vector(pieces[g][m]))/kol[g])
            buf+=1
    return out, kol, out2

def vectorsextender(data,kol):
    out = np.zeros(shape=(np.sum(kol),512))
    out2 = np.zeros(shape=(np.sum(kol)))
    buf=0
    for g in range(data.shape[0]):
        for k in range(kol[g]):
            out[buf]+=data[g]
            #out2[buf]+=np.sum(data[g])/512
            buf+=1
    return out#, out2

def positioncreator(kol):
    out = np.zeros(shape=(np.sum(kol)))
    buf=0
    for g in range(len(kol)):
        out[buf:buf+kol[g]]+=np.arange(1,kol[g]+1)
    return out
    

In [2]:
from keras.models import load_model
model = load_model ('D:/decoder/models/sen_to_word0.h5')

OSError: Unable to open file (unable to open file: name = 'D:/decoder/models/sen_to_word0.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
start=1
n=0
e=1
for d in range(12):
    print('Epoch: '+str(d)+'\n'+"**********************************\n")
    for i in range(200):
        vectors_train = np.load('C:/Users/DNS/Desktop/Progi/Python/NeuralNetworks/Курсач/data/data3000/news_vectors'+str(999-i)+'.npy')
        #print(start)
        texts = textreader(start)
        start+=500
        #print(len(texts))

        for j in range(5):
            tt, kol, garb = word_vectors_creator(texts[0+j*100:(1+j)*100])
            vtt = vectorsextender(vectors_train[0+j*100:(1+j)*100], kol)
            pos = positioncreator(kol)

            if n%20==0:
                model.fit(
                    x=[vtt,
                       pos],
                    y = [tt,
                        garb],
                    epochs=e, initial_epoch=n,
                    validation_split=0.05,
                    verbose=1,
                    batch_size=4
                )
                if n%100==0:
                    model.save('C:/Users/DNS/Desktop/Progi/Python/NeuralNetworks/Курсач/models/sen_to_word'+str(d)+'_'+str(i)+'.h5')
            else:
                model.fit(
                    x=[vtt,
                       pos],
                    y = [tt,
                        garb],
                    epochs=e, initial_epoch=n,
                    validation_split=0.05,
                    verbose=0,
                    batch_size=4
                )
            n+=1
            e+=1
    model.save('C:/Users/DNS/Desktop/Progi/Python/NeuralNetworks/Курсач/models/sen_to_word'+str(d)+'.h5')
    print('\n\n\n')

Epoch: 0
**********************************

Train on 1803 samples, validate on 95 samples
Epoch 1/1
1803/1803 [==============================] - 18s 10ms/step - loss: 0.3509 - Encoder-Output_loss: 0.2642 - Encoder-Garbedge_loss: 0.0867 - val_loss: 0.3242 - val_Encoder-Output_loss: 0.2470 - val_Encoder-Garbedge_loss: 0.0771
Train on 2162 samples, validate on 114 samples
Epoch 21/21
2162/2162 [==============================] - 17s 8ms/step - loss: 0.2743 - Encoder-Output_loss: 0.2425 - Encoder-Garbedge_loss: 0.0319 - val_loss: 0.2901 - val_Encoder-Output_loss: 0.2400 - val_Encoder-Garbedge_loss: 0.0500
Train on 2220 samples, validate on 117 samples
Epoch 41/41
2220/2220 [==============================] - 18s 8ms/step - loss: 0.2667 - Encoder-Output_loss: 0.2384 - Encoder-Garbedge_loss: 0.0283 - val_loss: 0.2649 - val_Encoder-Output_loss: 0.2242 - val_Encoder-Garbedge_loss: 0.0408
Train on 2319 samples, validate on 123 samples
Epoch 61/61
2319/2319 [==============================] - 18s 

2249/2249 [==============================] - 18s 8ms/step - loss: 0.2706 - Encoder-Output_loss: 0.2407 - Encoder-Garbedge_loss: 0.0299 - val_loss: 0.3054 - val_Encoder-Output_loss: 0.2415 - val_Encoder-Garbedge_loss: 0.0639
Train on 1990 samples, validate on 105 samples
Epoch 581/581
1990/1990 [==============================] - 16s 8ms/step - loss: 0.2776 - Encoder-Output_loss: 0.2429 - Encoder-Garbedge_loss: 0.0347 - val_loss: 0.2974 - val_Encoder-Output_loss: 0.2457 - val_Encoder-Garbedge_loss: 0.0518
Train on 3137 samples, validate on 166 samples
Epoch 601/601
3137/3137 [==============================] - 25s 8ms/step - loss: 0.2661 - Encoder-Output_loss: 0.2401 - Encoder-Garbedge_loss: 0.0259 - val_loss: 0.2959 - val_Encoder-Output_loss: 0.2601 - val_Encoder-Garbedge_loss: 0.0358
Train on 1945 samples, validate on 103 samples
Epoch 621/621
1945/1945 [==============================] - 16s 8ms/step - loss: 0.2716 - Encoder-Output_loss: 0.2385 - Encoder-Garbedge_loss: 0.0331 - val_loss

2432/2432 [==============================] - 19s 8ms/step - loss: 0.2639 - Encoder-Output_loss: 0.2375 - Encoder-Garbedge_loss: 0.0264 - val_loss: 0.2597 - val_Encoder-Output_loss: 0.2336 - val_Encoder-Garbedge_loss: 0.0261
Train on 2475 samples, validate on 131 samples
Epoch 1141/1141
2475/2475 [==============================] - 19s 8ms/step - loss: 0.2692 - Encoder-Output_loss: 0.2399 - Encoder-Garbedge_loss: 0.0293 - val_loss: 0.3076 - val_Encoder-Output_loss: 0.2490 - val_Encoder-Garbedge_loss: 0.0586
Train on 2011 samples, validate on 106 samples
Epoch 1161/1161
2011/2011 [==============================] - 16s 8ms/step - loss: 0.2673 - Encoder-Output_loss: 0.2391 - Encoder-Garbedge_loss: 0.0282 - val_loss: 0.3051 - val_Encoder-Output_loss: 0.2417 - val_Encoder-Garbedge_loss: 0.0634
Train on 2217 samples, validate on 117 samples
Epoch 1181/1181
2217/2217 [==============================] - 17s 8ms/step - loss: 0.2856 - Encoder-Output_loss: 0.2480 - Encoder-Garbedge_loss: 0.0376 - va

2672/2672 [==============================] - 21s 8ms/step - loss: 0.2625 - Encoder-Output_loss: 0.2357 - Encoder-Garbedge_loss: 0.0268 - val_loss: 0.2720 - val_Encoder-Output_loss: 0.2458 - val_Encoder-Garbedge_loss: 0.0262
Train on 1444 samples, validate on 76 samples
Epoch 1701/1701
1444/1444 [==============================] - 11s 8ms/step - loss: 0.2659 - Encoder-Output_loss: 0.2298 - Encoder-Garbedge_loss: 0.0361 - val_loss: 0.2772 - val_Encoder-Output_loss: 0.2193 - val_Encoder-Garbedge_loss: 0.0578
Train on 1895 samples, validate on 100 samples
Epoch 1721/1721
1895/1895 [==============================] - 15s 8ms/step - loss: 0.2675 - Encoder-Output_loss: 0.2346 - Encoder-Garbedge_loss: 0.0329 - val_loss: 0.2840 - val_Encoder-Output_loss: 0.2231 - val_Encoder-Garbedge_loss: 0.0610
Train on 2675 samples, validate on 141 samples
Epoch 1741/1741
2675/2675 [==============================] - 21s 8ms/step - loss: 0.2557 - Encoder-Output_loss: 0.2319 - Encoder-Garbedge_loss: 0.0239 - val

2713/2713 [==============================] - 21s 8ms/step - loss: 0.2713 - Encoder-Output_loss: 0.2407 - Encoder-Garbedge_loss: 0.0306 - val_loss: 0.2819 - val_Encoder-Output_loss: 0.2219 - val_Encoder-Garbedge_loss: 0.0600
Train on 3341 samples, validate on 176 samples
Epoch 2261/2261
3341/3341 [==============================] - 26s 8ms/step - loss: 0.2511 - Encoder-Output_loss: 0.2302 - Encoder-Garbedge_loss: 0.0209 - val_loss: 0.2727 - val_Encoder-Output_loss: 0.2278 - val_Encoder-Garbedge_loss: 0.0449
Train on 2346 samples, validate on 124 samples
Epoch 2281/2281
2346/2346 [==============================] - 18s 8ms/step - loss: 0.2722 - Encoder-Output_loss: 0.2414 - Encoder-Garbedge_loss: 0.0308 - val_loss: 0.3150 - val_Encoder-Output_loss: 0.2522 - val_Encoder-Garbedge_loss: 0.0628
Train on 1875 samples, validate on 99 samples
Epoch 2301/2301
1875/1875 [==============================] - 15s 8ms/step - loss: 0.2848 - Encoder-Output_loss: 0.2458 - Encoder-Garbedge_loss: 0.0390 - val

In [6]:
model.save('D:/decoder/models/sen_to_word'+str(i)+'.h5')

In [ ]:
start=1
n=0
e=1
for d in range(11,20):
    print('Epoch: '+str(d)+'\n'+"**********************************\n")
    for i in range(200):
        vectors_train = np.load('D:/decoder/data/data3000/news_vectors'+str(999-i)+'.npy')
        #print(start)
        texts = textreader(start)
        start+=500
        #print(len(texts))

        for j in range(5):
            tt, kol = word_vectors_creator(texts[0+j*100:(1+j)*100])
            vtt = vectorsextender(vectors_train[0+j*100:(1+j)*100], kol)
            pos = positioncreator(kol)

            if n%100==0:
                model.fit(
                    x=[vtt,
                       pos],
                    y = tt,
                    epochs=e, initial_epoch=n,
                    validation_split=0.05,
                    verbose=1
                    #batch_size=6
                )
            else:
                model.fit(
                    x=[vtt,
                       pos],
                    y = tt,
                    epochs=e, initial_epoch=n,
                    validation_split=0.05,
                    verbose=0
                    #batch_size=6
                )
            n+=1
            e+=1
    model.save('D:/decoder/models/sen_to_word'+str(d)+'.h5')
    print('\n\n\n')